In [1]:
# Importing Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from pyomo.environ import *

In [2]:
from __future__ import division
from pyomo.opt import SolverFactory

In [3]:
# Reading the data from Excel workbook
InputData = "Staffing+Data.xlsx"
StaffAv = pd.read_excel(InputData, sheet_name="StaffAvailability")
Cost = pd.read_excel(InputData, sheet_name="Cost")
transCost = pd.read_excel(InputData, sheet_name="ServiceRate")
Demand_data = pd.read_excel(InputData, sheet_name="DemandData")

In [ ]:
print('Staff Availability')
print(StaffAv.head(),'\n')
print('Cost')
print(Cost.head(),'\n')
print('Service Rate')
print(transCost.head(),'\n')
print('Demand Data')
print(Demand_data.head())

## Data pre-processing 

In [5]:
# Create the required Python data structures for indexes and parameters
#Create Demand dict
Demand = Demand_data.set_index(['State','Month'])['Demand'].to_dict()

#Create full time employee salary dict
FTE_Salary = Cost.set_index(['State','Month'])['MonthlySalary'].to_dict()

#Create unit out source cost dict
UnitOutSourceCost = Cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

In [ ]:
print(Demand)
print('################################')
print(FTE_Salary)
print('################################')
print(UnitOutSourceCost)

In [7]:
# Number of Applications when Employee working full time
FTE_AppServeRate = transCost.iloc[0,0]
FTE_AppServeRate

40

In [8]:
# Staff Availability for serving the Insurance Applications

StaffAv_LB = StaffAv.set_index(['State','Month'])['LB'].to_dict()
StaffAv_UB = StaffAv.set_index(['State','Month'])['UB'].to_dict()
StaffAvPer = StaffAv.set_index(['State','Month'])['StaffAvPer'].to_dict()

## Indexes

In [ ]:
BankLoc = list(Cost['State'].unique())
BankLoc

In [ ]:
Month = list(Cost['Month'].unique())
Month

# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



*Answered in the report*

**Note:** You can write the mathematical formulation directly in your report.

# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [11]:
# Creating a model instance
model = ConcreteModel()

In [12]:
# Define Pyomo sets and Parameters
model.bankloc = Set(initialize = BankLoc, doc='State')
model.month = Set(initialize = Month, doc='Month')

In [ ]:
# Parameters -> Demand, FTE Salaries, Outsourcing cost, Average Staff Availability
model.demand = Param(model.bankloc, model.month, initialize=Demand, doc='Demand')
model.salary = Param(model.bankloc, model.month, initialize=FTE_Salary, doc='FTE_Salary')
model.unitcost = Param(model.bankloc, model.month, initialize=UnitOutSourceCost, doc='UnitOutSourceCost')
model.staffavper = Param(model.bankloc, model.month, initialize=StaffAvPer, doc='StaffAvPercent')

In [14]:
# Scalar Application Serve Rate when working full time(100%)
model.fteapprate = Param(initialize=FTE_AppServeRate, doc='FTE App Serve Rate')

In [15]:
# Parameter for no of applications that can be processed for the given staff availaibility
def fteapp_staffav(model, loc, m):
    return model.fteapprate * model.staffavper[loc,m]

model.fteappno = Param(model.bankloc, model.month, initialize=fteapp_staffav, doc='No of FTE App as per StaffAvPer')

In [16]:
# Decision variables
model.fte = Var(model.bankloc,model.month,domain = NonNegativeReals, doc='No of FTE')
model.outsourced = Var(model.bankloc,model.month, domain = NonNegativeIntegers, doc='No of Outsourced applications')

In [17]:
# Constraints

#Constraint 1 -> Demand constraints
def demand_constraint(model,loc,m):
    return model.fte[loc,m]*model.fteappno[loc,m] + model.outsourced[loc,m] == model.demand[loc,m]

model.demand_meet = Constraint(model.bankloc, model.month, rule = demand_constraint)

# Constraint 2 --> Regulatory Constraints on outsourced applications
# State A --> 30%
# State B --> 40%
# State C --> No limit

Max_outsourced = {'A':0.3,'B':0.4,'C':1.0}

def outsourced_limit(model,loc,m):
    return model.outsourced[loc,m] <= Max_outsourced[loc]*model.demand[loc,m]

model.outsourced_lim = Constraint(model.bankloc, model.month, rule = outsourced_limit)

In [18]:
# Objective function
def obj_rule(model):
    return sum(model.fte[loc,m]*model.salary[loc,m] for loc in model.bankloc for m in model.month) + \
           sum(model.unitcost[loc,m]*model.outsourced[loc,m] for loc in model.bankloc for m in model.month)

model.cost = Objective(rule = obj_rule, sense=minimize, doc="Cost of total applications")

In [ ]:
# Invoking the solver
result = SolverFactory('glpk').solve(model)
result.write()

In [ ]:
# Print the value of the objective function
model.cost()

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [ ]:
# Creating dataframe for the results
# FTE_StaffAvPer containing FTE and number of applications outsourced
FTE_StaffAvPer = []
for loc in BankLoc:
    for m in Month:
        v = model.fte[loc,m].value
        w = model.outsourced[loc,m].value
        d = model.demand[loc,m]
        sa = model.staffavper[loc,m]
        FTE_StaffAvPer.append([loc,m,d,sa,v,w])
        
FTE_StaffAvPer = pd.DataFrame(FTE_StaffAvPer, columns=['State','Month','Demand','Staff Availability','FTE','App_Outsourced'])
FTE_StaffAvPer

In [ ]:
# Some obtained results --> Average FTE per month across all states
TotalFTE = FTE_StaffAvPer['FTE'].sum()
AverageFTEPerMonth = (TotalFTE/(12*3)).round(2)  #Since we are looking for all states, 3 has been multiplied in the denominator
AverageFTEPerMonth

In [ ]:
#Total Application Demand
TotalAppDemand = Demand_data['Demand'].sum()
TotalAppDemand

In [ ]:
# Percentage of Outsourced Applications
PercentageOutsourced = ((FTE_StaffAvPer['App_Outsourced'].sum()/TotalAppDemand)*100).round(2)
PercentageOutsourced

In [ ]:
TotalCost = model.cost()
AverageCostPerAppl = (TotalCost / TotalAppDemand).round(2)
AverageCostPerAppl

# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [ ]:
# Creating a model instance
model_lb =  ConcreteModel()

In [ ]:
# Define Pyomo sets and Parameters
model_lb.bankloc = Set(initialize = BankLoc, doc='State')
model_lb.month = Set(initialize = Month, doc='Month')

In [ ]:
# Parameters -> Demand, FTE Salaries, Outsourcing cost, Staff Availability Lower Bound
model_lb.demand = Param(model_lb.bankloc, model_lb.month, initialize=Demand, doc='Demand')
model_lb.salary = Param(model_lb.bankloc, model_lb.month, initialize=FTE_Salary, doc='FTE_Salary')
model_lb.unitcost = Param(model_lb.bankloc, model_lb.month, initialize=UnitOutSourceCost, doc='UnitOutSourceCost')
model_lb.staffavlb = Param(model_lb.bankloc, model_lb.month, initialize=StaffAv_LB, doc='StaffAvLowerBound')

In [ ]:
# Scalar Application Serve Rate when working full time(100%)
model_lb.fteapprate = Param(initialize=FTE_AppServeRate, doc='FTE App Serve Rate')

In [ ]:
# Parameter for no of applications that can be processed for the given staff availaibility
def fteapp_staffavlb(model, loc, m):
    return model_lb.fteapprate * model_lb.staffavlb[loc,m]

model_lb.fteappno = Param(model_lb.bankloc, model_lb.month, initialize=fteapp_staffavlb, doc='No of FTE App as per StaffAv_LB')

In [ ]:
# Decision variables
model_lb.fte = Var(model_lb.bankloc,model_lb.month,domain = NonNegativeReals, doc='No of FTE')
model_lb.outsourced = Var(model_lb.bankloc,model_lb.month, domain = NonNegativeIntegers, doc='No of Outsourced applications')

In [ ]:
# Constraints

#Constraint 1 -> Demand constraints
def demand_constraint(model,loc,m):
    return model_lb.fte[loc,m]*model_lb.fteappno[loc,m] + model_lb.outsourced[loc,m] == model_lb.demand[loc,m]

model_lb.demand_meet = Constraint(model_lb.bankloc, model_lb.month, rule = demand_constraint)

# Constraint 2 ->Regulatory Constraints on outsourced applications
# State A --> 30%
# State B --> 40%
# State C --> No limit

Max_outsourced = {'A':0.3,'B':0.4,'C':1.0}

def outsourced_limit(model,loc,m):
    return model_lb.outsourced[loc,m] <= Max_outsourced[loc]*model_lb.demand[loc,m]

model_lb.outsourced_lim = Constraint(model_lb.bankloc, model_lb.month, rule = outsourced_limit)

In [ ]:
# Objective function
def obj_rule(model):
    return sum(model_lb.fte[loc,m]*model_lb.salary[loc,m] for loc in model_lb.bankloc for m in model_lb.month) + \
           sum(model_lb.unitcost[loc,m]*model_lb.outsourced[loc,m] for loc in model_lb.bankloc for m in model_lb.month)

model_lb.cost = Objective(rule = obj_rule)

In [ ]:
# Invoking the solver
result = SolverFactory('glpk').solve(model_lb)
result.write()

In [ ]:
# Print the value of the objective function
model_lb.cost()

**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [ ]:
# Creating dataframe for the results
# FTE_StaffAv_LB containing FTE and number of applications outsourced
FTE_StaffAv_LB = []
for loc in BankLoc:
    for m in Month:
        v = model_lb.fte[loc,m].value
        w = model_lb.outsourced[loc,m].value
        d = model.demand[loc,m]
        sa = model.staffavper[loc,m]
        FTE_StaffAv_LB.append([loc,m,d,sa,v,w])
        
FTE_StaffAv_LB = pd.DataFrame(FTE_StaffAv_LB, columns=['State','Month','Demand','Staff Availability','FTE','App_Outsourced'])
FTE_StaffAv_LB

#### 3.1.2 Percentage of outsourced applications 

In [ ]:
# write your code here
PercentageOutsourced_LB = ((FTE_StaffAv_LB['App_Outsourced'].sum()/TotalAppDemand)*100).round(2)
PercentageOutsourced_LB

#### 3.1.3 Average cost per application

In [ ]:
# write your code here
TotalCost_LB = model_lb.cost()
AverageCostPerAppl_LB = (TotalCost_LB / TotalAppDemand).round(2)
AverageCostPerAppl_LB

### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members


In [ ]:
# Creating a model instance
model_ub =  ConcreteModel()

In [ ]:
# Define Pyomo sets and Parameters
model_ub.bankloc = Set(initialize = BankLoc, doc='State')
model_ub.month = Set(initialize = Month, doc='Month')

In [ ]:
# Parameters -> Demand, FTE Salaries, Outsourcing cost, Staff Availability Upper Bound
model_ub.demand = Param(model_ub.bankloc, model_ub.month, initialize=Demand, doc='Demand')
model_ub.salary = Param(model_ub.bankloc, model_ub.month, initialize=FTE_Salary, doc='FTE_Salary')
model_ub.unitcost = Param(model_ub.bankloc, model_ub.month, initialize=UnitOutSourceCost, doc='UnitOutSourceCost')
model_ub.staffavub = Param(model_ub.bankloc, model_ub.month, initialize=StaffAv_UB, doc='StaffAvUpperBound')

In [ ]:
# Scalar Application Serve Rate when working full time(100%)
model_ub.fteapprate = Param(initialize=FTE_AppServeRate, doc='FTE App Serve Rate')

In [ ]:
# Parameter for no of applications that can be processed for the given staff availaibility
def fteapp_staffavub(model, loc, m):
    return model_ub.fteapprate * model_ub.staffavub[loc,m]

model_ub.fteappno = Param(model_ub.bankloc, model_ub.month, initialize=fteapp_staffavub, doc='No of FTE App as per StaffAv_UB')

In [ ]:
# Decision variables
model_ub.fte = Var(model_ub.bankloc,model_ub.month,domain = NonNegativeReals, doc='No of FTE')
model_ub.outsourced = Var(model_ub.bankloc,model_ub.month, domain = NonNegativeIntegers, doc='No of Outsourced applications')

In [ ]:
# Constraints

#Constraint 1 -> Demand constraints
def demand_constraint(model,loc,m):
    return model_ub.fte[loc,m]*model_ub.fteappno[loc,m] + model_ub.outsourced[loc,m] == model_ub.demand[loc,m]

model_ub.demand_meet = Constraint(model_ub.bankloc, model_ub.month, rule = demand_constraint)

# Constraint 2 -> Regulatory Constraints on outsourced applications
# State A --> 30%
# State B --> 40%
# State C --> No limit

Max_outsourced = {'A':0.3,'B':0.4,'C':1.0}

def outsourced_limit(model,loc,m):
    return model_ub.outsourced[loc,m] <= Max_outsourced[loc]*model_ub.demand[loc,m]

model_ub.outsourced_lim = Constraint(model_ub.bankloc, model_ub.month, rule = outsourced_limit)

In [ ]:
# Objective function
def obj_rule(model):
    return sum(model_ub.fte[loc,m]*model_ub.salary[loc,m] for loc in model_ub.bankloc for m in model_ub.month) + \
           sum(model_ub.unitcost[loc,m]*model_ub.outsourced[loc,m] for loc in model_ub.bankloc for m in model_ub.month)

model_ub.cost = Objective(rule = obj_rule)

In [ ]:
# Invoking the solver
result = SolverFactory('glpk').solve(model_ub)
result.write()

In [ ]:
# Print the value of the objective function
model_ub.cost()

**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [ ]:
# Creating dataframe for the results
# FTE_StaffAv_UB containing FTE and number of applications outsourced
FTE_StaffAv_UB = []
for loc in BankLoc:
    for m in Month:
        v = model_ub.fte[loc,m].value
        w = model_ub.outsourced[loc,m].value
        d = model.demand[loc,m]
        sa = model.staffavper[loc,m]
        FTE_StaffAv_UB.append([loc,m,d,sa,v,w])
        
FTE_StaffAv_UB = pd.DataFrame(FTE_StaffAv_UB, columns=['State','Month','Demand','Staff Availability','FTE','App_Outsourced'])
FTE_StaffAv_UB

#### 3.2.2 Percentage of outsourced applications

In [ ]:
# write your code here
PercentageOutsourced_UB = ((FTE_StaffAv_UB['App_Outsourced'].sum()/TotalAppDemand)*100).round(2)
PercentageOutsourced_UB

#### 3.2.3 Average cost per application

In [ ]:
# write your code here
TotalCost_UB = model_ub.cost()
AverageCostPerAppl_UB = (TotalCost_UB / TotalAppDemand).round(2)
AverageCostPerAppl_UB

# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.

In [ ]:
df = FTE_StaffAvPer.copy()
df

In [ ]:
df["FTE_Percent"] = (df["FTE"]*df["Staff Availability"]*40)*100/df["Demand"]
df["App_Outsourced_Percent"] = df["App_Outsourced"]*100/df["Demand"]

In [ ]:
df.head()

In [ ]:
df.set_index(['Month', 'State'], inplace=True)

In [ ]:
df = df.unstack(level=-1)

In [ ]:
df0 = df[["FTE_Percent", "App_Outsourced_Percent"]]
df0.head()

In [ ]:
ordered= ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
df0 = df0.reindex(ordered)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(18.5, 10.5)
(df0['FTE_Percent']+df0['App_Outsourced_Percent']).plot(kind='bar', color=[(0.05, 0.80, 0.89), (0.0, 0.6, 0.98),(0.12, 0.3, 0.7)], rot=0, ax=ax)
df0['App_Outsourced_Percent'].plot(kind='bar', color=[(0.89, 0.10, 0.10), (0.9, 0.3, 0.8), (0.54, 0.1, 0.6)], rot=0, ax=ax)

legend_labels = [f'{state} ({param})' for param, state in df0.columns]
ax.legend(legend_labels, loc="best")

rects = ax.patches
labels = ['A'] * 12 + ['B']*12 + ['C']*12

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width() / 2, height + 0.1, label, ha="center", va="bottom"
    )

plt.ylabel("FTE percent v/s Outsourced percent")
plt.title("Month and Statewise comparison of FTE v/s Outsourced")
plt.show()

## State Wise analysis

In [ ]:
state_df = FTE_StaffAvPer.copy()
state_df.head()

In [ ]:
state_df["FTE_App"] = state_df["FTE"]*state_df["Staff Availability"]*FTE_AppServeRate
state_df.head()

In [ ]:
state_df = state_df.groupby("State")[["Demand","FTE","FTE_App", "App_Outsourced"]].sum()
state_df.head()

In [ ]:
state_df["FTE_Percent"] = state_df["FTE_App"]*100/state_df["Demand"]
state_df["App_Outsourced_Percent"] = state_df["App_Outsourced"]*100/state_df["Demand"]
state_df

In [ ]:
cost_per_month_state = {'A':5000,'B':4583.333333,'C':4416.666666}
state_df["FTE Cost per month"] = state_df.index.map(cost_per_month_state)
state_df

In [ ]:
outsource_cost_per_app = {'A':180,'B':150,'C':160}
state_df["Outsource Cost per App"] = state_df.index.map(outsource_cost_per_app)
state_df

In [ ]:
state_df["Total Cost"] = state_df["FTE"]*state_df["FTE Cost per month"] + state_df["App_Outsourced"] * state_df["Outsource Cost per App"]
state_df

In [ ]:
state_df["Total Cost per App"] = state_df["Total Cost"]/state_df["Demand"]
state_df

In [ ]:
state_df.plot.barh(y="Total Cost per App")
plt.xlabel("Cost per Application")
plt.title("State wise Cost per Application")
plt.show()

In [ ]:
state_df[["FTE_Percent","App_Outsourced_Percent"]].plot(kind="bar",stacked=True)
plt.legend(loc='best')
plt.xticks(rotation=360)
plt.ylabel("FTE_percent v/s Outsourced Percent")
plt.title("Statewise FTE percent v/s Outsourced Percent")
plt.show()

### Insights
- From the Best case and Worst case analysis we can conclude that, less the availability of staff which means more the percentage of outsourced applications, the company has to incur more cost.
- In Best case scenario the total cost decreased by 1.44 million dollars while for worst case it increased by 1.63 million dollars
- From the state wise analysis we see state A has the highest cost per application while state C has the lowest cost per application.
- Also, we see that state A has the lowest percentage of outsourced applications as compared to other states.